In [2]:
import pandas as pd
import re


df = pd.read_csv('linkedin_jobs_germany_enriched.csv')


df = df.drop_duplicates(subset=['Link'])


def clean_title(title):
    if not isinstance(title, str):
        return ''
  
    title = re.sub(r'\s*\((?:m/w/d|m/f/x|gn|all genders|alle|div|m|w|d)\)', '', title, flags=re.IGNORECASE)
    title = re.sub(r'\s+', ' ', title)
    return title.strip()

df['Title'] = df['Title'].apply(clean_title)

df['Seniority'] = df['Seniority'].str.lower().str.strip()
df['Seniority'] = df['Seniority'].replace({'werkstudent': 'werkstudent', 'junior': 'junior', 'senior': 'senior', 'praktikum': 'praktikum', 'student': 'werkstudent'})

df['Lang'] = df['Lang'].str.lower().str.strip()
df['Lang'] = df['Lang'].replace({'deutsch': 'de', 'german': 'de', 'englisch': 'en', 'english': 'en', 'en-de': 'mixed', 'de-en': 'mixed'})


df['Company'] = df['Company'].str.strip().str.title()


bool_cols = [col for col in df.columns if col.startswith('Has_') or col in ['Remote']]
for col in bool_cols:
    df[col] = df[col].apply(lambda x: True if str(x).strip().lower() in ['true', '1', 'yes'] else False)

df = df.dropna(subset=['Title', 'Company'], how='all')

df = df.dropna(axis=1, how='all')
df = df.loc[:, (df != False).any(axis=0)]

df.to_csv('linkedin_jobs_germany_clean.csv', index=False)
print("✅linkedin_jobs_germany_clean.csv")


✅ Чистка закончена. Файл сохранён как linkedin_jobs_germany_clean.csv


In [3]:
import pandas as pd

df = pd.read_csv('linkedin_jobs_germany_clean.csv')

df = df[df['Title'].str.count(r'[A-Za-zА-Яа-я]') > 5]


df = df[~df['Title'].str.contains(r'^\*+|/|\\|^[\W_]+$', regex=True)]

df['Title'] = df['Title'].str.title().str.strip()

df.to_csv('linkedin_jobs_germany_clean2.csv', index=False)
print("✅")


✅ Грязные строки и мусорные вакансии удалены. Файл сохранён.
